In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error
import pickle

# LinearRegression

Observations from initial linear regression


1. The scores are more important than the trends

2. All combined is the best model

3. Only trends is worse

4. Only scores is close

5. There is no variance. (So increase features)

In [19]:
def mape(y_true,y_predict):
    return np.sum((np.abs(y_true-y_predict)/y_true)*100)/len(y_true)

In [20]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
feature_df = pd.read_csv('csv_data/feature_first_innings.csv',parse_dates=['match_date'],date_parser=custom_date_parser)

In [21]:
feature_df['team'].unique()

array(['Australia', 'Sri Lanka', 'Zimbabwe', 'South Africa', 'India',
       'New Zealand', 'England', 'Pakistan', 'West Indies', 'Bangladesh',
       'Ireland', 'Scotland', 'Afghanistan', 'United Arab Emirates',
       'Hong Kong', 'Papua New Guinea', 'United States of America'],
      dtype=object)

In [22]:
# cutoff_start_year = '2014'
# cutoff_start_date = datetime.strptime(cutoff_start_year+'-01-01','%Y-%m-%d')
# #feature_df=feature_df[feature_df['match_date']>cutoff_start_date]
# feature_df=feature_df[feature_df['team']=='India']
# feature_df=feature_df[feature_df['match_date']>cutoff_start_date]
#feature_df = feature_df[feature_df['noise']==False]

In [23]:
feature_df.columns

Index(['match_id', 'match_date', 'team', 'opponent', 'location', 'team_score',
       'opponent_score', 'opponent_base', 'opponent_trend',
       'opponent_trend_predict', 'opponent_mean', 'location_base',
       'location_trend', 'location_trend_predict', 'location_mean',
       'current_base', 'current_trend', 'current_trend_predict',
       'current_mean', 'batsman_mean', 'batsman_max', 'bowler_mean',
       'bowler_max', 'is_train', 'noise', 'runs_scored'],
      dtype='object')

In [24]:
feature_df.dropna(inplace=True)
feature_df.isnull().sum()

match_id                  0
match_date                0
team                      0
opponent                  0
location                  0
team_score                0
opponent_score            0
opponent_base             0
opponent_trend            0
opponent_trend_predict    0
opponent_mean             0
location_base             0
location_trend            0
location_trend_predict    0
location_mean             0
current_base              0
current_trend             0
current_trend_predict     0
current_mean              0
batsman_mean              0
batsman_max               0
bowler_mean               0
bowler_max                0
is_train                  0
noise                     0
runs_scored               0
dtype: int64

In [25]:
# feature_columns = ['team_score',
#        'opponent_score', 'opponent_base', 'opponent_trend',
#        'opponent_trend_predict', 'opponent_mean', 'location_base',
#        'location_trend', 'location_trend_predict', 'location_mean',
#        'current_base', 'current_trend', 'current_trend_predict',
#        'current_mean', 'batsman_mean', 'batsman_max', 'bowler_mean',
#        'bowler_max']
pvalue selection : selected x1,x2,x7,x10,x15,x16,x17

feature_columns = ['team_score', 'opponent_score', 'location_base', 'location_mean','batsman_mean', 'batsman_max', 'bowler_mean']
       


target_column = ['runs_scored']

In [26]:
feature_df_train = feature_df[feature_df['is_train']==True]
feature_df_test = feature_df[feature_df['is_train']==False]

In [27]:
scaler = StandardScaler()
y_scaler = StandardScaler()
x_train = scaler.fit_transform(feature_df_train[feature_columns])
y_train = feature_df_train[target_column]
y_train_sc = y_scaler.fit_transform(feature_df_train[target_column])

x_test = scaler.transform(feature_df_test[feature_columns])
y_test = feature_df_test[target_column]
y_test_sc = y_scaler.transform(feature_df_test[target_column])




# x_train = np.array(feature_df_train[feature_columns])
# y_train = np.array(feature_df_train[target_column])

# x_test = np.array(feature_df_test[feature_columns])
# y_test = np.array(feature_df_test[target_column])




In [11]:
lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

In [12]:
y_train_predict = lr.predict(x_train)
y_test_predict = lr.predict(x_test)

In [13]:
mean_absolute_error(y_train,y_train_predict)

42.05695001923458

In [14]:
mean_absolute_error(y_test,y_test_predict)

46.288914601173744

In [15]:
mape(np.array(y_train),np.array(y_train_predict))

19.921629717476897

In [16]:
mape(np.array(y_test),np.array(y_test_predict))

22.4293955796168

In [17]:
mean_squared_error(y_test,y_test_predict)

3441.572405244475

In [161]:
#pickle.dump(lr,open('first_innings_linear_regression.pkl','wb'))

In [164]:
#pickle.dump(scaler,open('first_innings_linear_regression_scaler.pkl','wb'))

In [163]:
result_df_train=pd.DataFrame(feature_df_train)
result_df_test =pd.DataFrame(feature_df_test)
result_df_train['first_innings_prediction']=y_train_predict
result_df_test['first_innings_prediction']=y_test_predict

result_df_train.to_csv('csv_data/result_df_tran.csv',index=False)
result_df_test.to_csv('csv_data/result_df_test.csv',index=False)

In [67]:
y_test

,runs_scored
723,371
724,319
725,364
726,266
727,203
...,...
827,294
828,231
829,302
830,374


In [68]:
compare_df = pd.DataFrame()
compare_df['actual']=y_test['runs_scored']
compare_df['predicted']=y_test_predict.reshape(-1)

In [69]:
compare_df['percentage_error']=abs(compare_df['actual']-compare_df['predicted'])/compare_df['actual']

In [70]:
compare_df

,actual,predicted,percentage_error
723,371,287.470210,0.225148
724,319,276.350358,0.133698
725,364,294.862849,0.189937
726,266,297.507207,0.118448
727,203,246.928023,0.216394
...,...,...,...
827,294,235.756259,0.198108
828,231,249.803472,0.081400
829,302,262.209381,0.131757
830,374,278.053649,0.256541


In [71]:
compare_df['is_less']=compare_df['actual']>compare_df['predicted']

In [72]:
compare_df[compare_df['is_less']==True].shape,compare_df.shape

((65, 4), (109, 4))

In [73]:
result_df =pd.concat([feature_df_test,compare_df],axis=1)

In [74]:
result_df[result_df['percentage_error']<0.15]

,match_id,match_date,team,opponent,location,team_score,opponent_score,opponent_base,opponent_trend,opponent_trend_predict,...,batsman_max,bowler_mean,bowler_max,is_train,noise,runs_scored,actual,predicted,percentage_error,is_less
724,1153841,2019-01-05,New Zealand,Sri Lanka,Mount Maunganui,2.407534,1.809724,294.9,7.5,339.9,...,4.993700,2.810098,3.745236,False,False,319,319,276.350358,0.133698,True
726,1144156,2019-01-19,South Africa,Pakistan,Port Elizabeth,2.279248,2.745555,196.7,15.1,287.3,...,3.353053,3.327420,4.287456,False,False,266,266,297.507207,0.118448,False
729,1153693,2019-01-28,New Zealand,India,Mount Maunganui,2.407534,3.597041,212.0,9.8,270.8,...,4.993700,4.287744,6.077775,False,False,243,243,250.877114,0.032416,False
730,1144160,2019-01-30,Pakistan,South Africa,Cape Town,2.745555,2.279248,229.5,-5.1,198.9,...,4.962897,3.423204,4.023650,False,False,240,240,224.345770,0.065226,True
732,1153695,2019-02-03,India,New Zealand,Wellington,3.597041,2.407534,353.4,-31.0,167.4,...,5.731100,3.801929,4.644827,False,True,252,252,217.528811,0.136790,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,1185321,2020-03-07,Australia,South Africa,Potchefstroom,3.440821,2.292825,279.2,-13.6,197.6,...,5.071416,4.084507,4.482480,False,True,254,254,267.397895,0.052748,False
823,1183537,2020-03-13,Australia,New Zealand,Sydney,3.440821,3.156275,234.8,12.4,309.2,...,5.071416,4.830566,6.031234,False,False,258,258,260.952098,0.011442,False
825,1198248,2020-08-01,Ireland,England,Southampton,1.694051,3.682456,246.0,-21.9,136.5,...,3.056867,4.018376,5.344566,False,False,212,212,203.061060,0.042165,True
828,1198239,2020-09-13,England,Australia,Manchester,3.682456,3.440821,164.8,50.4,467.2,...,5.038464,5.402401,6.041895,False,False,231,231,249.803472,0.081400,False


In [75]:
for country in result_df['team'].unique():
    predicted_series=result_df[result_df['team']==country]['predicted']
    actual_series=result_df[result_df['team']==country]['actual']

    print(country,mape(np.array(actual_series),np.array(predicted_series)))

New Zealand 13.46871574437135
South Africa 21.536531467019987
Pakistan 31.83534224113266
India 24.34581476078308
Bangladesh 11.776267116083451
West Indies 13.066617441772507
Australia 13.155988629337028
Sri Lanka 21.50163433439312
Afghanistan 20.351210150310695
United Arab Emirates 83.43339095478932
Ireland 7.904914695676502
England 10.77402879188841
Papua New Guinea 12.136828649392898
Scotland 9.629465339140028
United States of America 471.30531818433514


In [76]:
for country in result_df['team'].unique():
    std = result_df[result_df['team']==country]['actual'].std()
    print(country,std)

New Zealand 54.58894009735288
South Africa 39.92373682290727
Pakistan 78.25059460192975
India 74.65657637210802
Bangladesh 47.44119166575253
West Indies 50.84271966424635
Australia 49.0285631035624
Sri Lanka 66.35565331810642
Afghanistan 35.7794168016566
United Arab Emirates nan
Ireland 51.05389309347525
England 51.46508083598573
Papua New Guinea nan
Scotland 15.556349186104045
United States of America nan


# lasso

In [77]:
from sklearn import linear_model

In [78]:
las = linear_model.Lasso(alpha=0.3)
las.fit(x_train,y_train_sc)

Lasso(alpha=0.3)

In [79]:
y_test_predict_ls = las.predict(x_test)
y_train_predict_ls = las.predict(x_train)

In [80]:
mape(np.array(y_train_sc).reshape(-1),np.array(y_train_predict_ls))

64.19679606050401

In [81]:
mape(np.array(y_test_sc).reshape(-1),np.array(y_test_predict_ls))

43.392242971798865

In [82]:
y_test_predict_original=y_scaler.inverse_transform(y_test_predict_ls)
#y_test_original = feature_df_test[target_column]

In [83]:
mape(np.array(y_test).reshape(-1),np.array(y_test_predict_original))

25.675439359472517

In [84]:
y_test_predict_ls

array([ 0.10423353,  0.07712387,  0.10423353,  0.02567081,  0.02276296,
       -0.04799696,  0.05441921,  0.0300646 ,  0.07107828,  0.11913245,
        0.06076495,  0.06076495,  0.10355943,  0.04720952, -0.05614589,
       -0.09070187,  0.16090654, -0.07370439,  0.06475524,  0.11214081,
        0.07999769, -0.06552653, -0.12200739,  0.01791544, -0.03948215,
       -0.03675088, -0.04976564, -0.1945434 , -0.06334171,  0.05619411,
       -0.02907993,  0.05914411,  0.30659055, -0.05004346,  0.03689991,
       -0.02557812,  0.07219484, -0.07861812,  0.12998278,  0.12806247,
        0.17092101,  0.01616185,  0.08220739, -0.07979251,  0.12273314,
        0.06413997,  0.08843853, -0.04662902, -0.10760601,  0.18640159,
        0.2494473 , -0.00334339, -0.08520959,  0.17180247, -0.00295773,
       -0.05978177, -0.10074244,  0.09849947,  0.05297882, -0.09701586,
        0.14889848, -0.07979251,  0.0707801 , -0.12003632,  0.18640159,
       -0.0400163 ,  0.13139447,  0.16662871,  0.07148172, -0.05

In [85]:
np.array(y_test).reshape(-1)

array([371, 319, 364, 266, 203, 164, 243, 240,  92, 252, 232, 226, 330,
       289, 236, 231, 250, 251, 223, 358, 216, 189, 272, 284, 266, 277,
       327, 110, 198, 381, 261, 327, 373, 247, 358, 292, 340, 210, 351,
       305, 311, 105, 207, 136, 330, 348, 244, 227, 288, 386, 352, 212,
       334, 397, 241, 381, 232, 224, 308, 285, 268, 203, 227, 243, 337,
       338, 314, 305, 311, 242, 315, 264, 239, 314, 238, 294, 205, 242,
       305, 297, 249, 220, 287, 387, 315, 237, 255, 340, 286, 258, 347,
       273, 256, 296,  35, 289, 345, 291, 322, 254, 258, 172, 212, 328,
       294, 231, 302, 374, 389])

# XGBOOST

In [86]:
import xgboost as xg

In [87]:
train_dmatrix = xg.DMatrix(data = x_train, label = y_train) 
test_dmatrix = xg.DMatrix(data = x_test, label = y_test) 

In [88]:
param = {"booster":"gblinear", "objective":"reg:linear"} 

In [89]:
xgb_r = xg.train(params = param, dtrain = train_dmatrix, num_boost_round = 10) 


[20:50:27] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[20:50:27] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


In [90]:
y_test_predict_xg = xgb_r.predict(test_dmatrix)
y_train_predict_xg = xgb_r.predict(train_dmatrix)

In [91]:
# y_train_predict_dt = dt.predict(x_train)
# y_test_predict_dt = dt.predict(x_test)

In [92]:
mean_absolute_error(np.array(y_test),np.array(y_test_predict_xg))

45.5171721572176

In [93]:
mape(np.array(y_test).reshape(-1),y_test_predict_xg)

22.65944162359071

In [94]:
mape(np.array(y_train).reshape(-1),y_train_predict_xg)

20.25278265779196

In [95]:
#y_test_predict_xg

# Randomforest

In [96]:
from sklearn.ensemble import RandomForestRegressor

In [97]:
rf = RandomForestRegressor(max_depth=8,n_estimators=50,criterion='mae')

In [98]:
rf.fit(x_train,y_train)

<ipython-input-98-895403b03771>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train,y_train)


RandomForestRegressor(criterion='mae', max_depth=8, n_estimators=50)

In [99]:
y_train_predict_rf=rf.predict(x_train)
y_test_predict_rf= rf.predict(x_test)

In [100]:
mape(np.array(y_test).reshape(-1),y_test_predict_rf)

23.281510548596753

In [101]:
mape(np.array(y_train).reshape(-1),y_train_predict_rf)

13.462300581977942

# statsmodel

In [28]:
import statsmodels.api as sm

In [29]:
model = sm.OLS(y_train, sm.add_constant(x_train)).fit()

In [30]:
y_test_predicted_sm=model.predict(sm.add_constant(x_test))

In [31]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            runs_scored   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     18.98
Date:                Fri, 01 Jan 2021   Prob (F-statistic):           7.27e-43
Time:                        23:13:15   Log-Likelihood:                -3916.2
No. Observations:                 723   AIC:                             7864.
Df Residuals:                     707   BIC:                             7938.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        255.8728      2.048    124.907      0.000     251.851     259.895
x1            -9.0921      3.352     -2.712      0.007     -15.673      -2.511
x2             7.4274      3.097      2.398      0.017       1.347      13.507
x3             1.3299      4.247      0.313      0.754      -7.008       9.668
x4             2.0869      7.099      0.294      0.769     -11.850      16.024
x5             0.4358      4.262      0.102      0.919      -7.931       8.803
x6             1.7270      1.567      1.102      0.271      -1.349       4.803
x7            12.4907      5.957      2.097      0.036       0.795      24.187
x8            14.4552      8.949      1.615      0.107      -3.115      32.025
x9            -8.0592      5.834     -1.381      0.168     -19.513       3.394
x10            2.6796      1.260      2.126      0.034       0.205       5.154
x11           20.1053     36.501      0.551      0.582     -51.557      91.768
x12           34.7721     62.954      0.552      0.581     -88.827     158.371
x13          -19.3525     36.608     -0.529      0.597     -91.226      52.521
x14            0.9323      1.737      0.537      0.592      -2.478       4.343
x15           55.4294      4.578     12.109      0.000      46.442      64.417
x16          -27.3444      3.874     -7.059      0.000     -34.950     -19.739
x17          -19.8676      4.620     -4.300      0.000     -28.938     -10.797
x18            6.3210      4.005      1.578      0.115      -1.541      14.183
==============================================================================
Omnibus:                       12.311   Durbin-Watson:                   1.850
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               13.775
Skew:                          -0.246   Prob(JB):                      0.00102
Kurtosis:                       3.464   Cond. No.                     9.12e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.62e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

selected x1,x2,x7,x10,x15,x16,x17

In [32]:
mape(np.array(y_test).reshape(-1),y_test_predicted_sm)


22.039536421362772

In [33]:
mean_absolute_error(np.array(y_test).reshape(-1),y_test_predicted_sm)

45.43723924019558

In [44]:
np.where(np.array(model.pvalues)<0.05)

(array([ 0,  1,  2,  7, 10, 15, 16, 17]),)

# with PCA

In [108]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

In [109]:
pca_scaler = StandardScaler()

In [110]:
pca = PCA(n_components=2)

In [111]:
x_train_pca=pca.fit_transform(pca_scaler.fit_transform(x_train))

In [112]:
lr_pca = LinearRegression()
lr_pca.fit(x_train_pca,y_train)

LinearRegression()

In [113]:
y_test_predict_pca =lr_pca.predict(pca.transform(pca_scaler.transform(x_test)))

In [114]:
mape(np.array(y_test),y_test_predict_pca)

24.557551133830028

In [115]:
#y_test_predict_pca.shape

In [116]:
#np.array(y_test).reshape(-1).shape

# SVM

In [117]:
from sklearn.svm import SVR

In [118]:
svr = SVR(C=0.5, epsilon=0.01)

In [119]:
svr.fit(x_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVR(C=0.5, epsilon=0.01)

In [120]:
y_test_predict_svr =svr.predict(x_test)

In [121]:
mape(np.array(y_test).reshape(-1),y_test_predict_svr)

25.846783163229134

# polynomial regression

In [122]:
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

In [132]:
poly = PolynomialFeatures(3)

In [133]:
x_train_poly = poly.fit_transform(x_train)

In [134]:
lr_poly = LinearRegression()

In [135]:
lr_poly.fit(x_train_poly,y_train)

LinearRegression()

In [136]:
y_test_predict_poly = lr_poly.predict(poly.transform(x_test))
y_train_predict_poly = lr_poly.predict(poly.transform(x_train))

In [137]:
mape(np.array(y_test),y_test_predict_poly)

187.18202146462477

In [138]:
mape(np.array(y_train),y_train_predict_poly)

3.0130001924180565

In [139]:
compare_df = pd.DataFrame()
compare_df['actual']=np.array(y_test).reshape(-1)
compare_df['predict']=np.array(y_test_predict_poly)

In [140]:
compare_df

,actual,predict
0,371,606.328315
1,319,489.073878
2,364,343.832440
3,266,344.225914
4,203,369.945787
...,...,...
104,294,520.875259
105,231,229.444440
106,302,295.823723
107,374,650.466732


In [141]:
np.array(y_test).reshape(-1).shape

(109,)

# Comparing trend predictions

In [142]:
feature_df.columns

Index(['match_id', 'match_date', 'team', 'opponent', 'location', 'team_score',
       'opponent_score', 'opponent_base', 'opponent_trend',
       'opponent_trend_predict', 'opponent_mean', 'location_base',
       'location_trend', 'location_trend_predict', 'location_mean',
       'current_base', 'current_trend', 'current_trend_predict',
       'current_mean', 'batsman_mean', 'batsman_max', 'bowler_mean',
       'bowler_max', 'is_train', 'noise', 'runs_scored'],
      dtype='object')

In [143]:
feature_df[['opponent_trend_predict','runs_scored']]

,opponent_trend_predict,runs_scored
0,377.0,271
1,165.0,249
2,302.0,219
3,136.0,290
4,205.0,119
...,...,...
827,245.3,294
828,467.2,231
829,389.4,302
830,311.8,374


In [144]:
opponent_trend_predict = np.array(feature_df['opponent_trend_predict'])
current_trend_predict = np.array(feature_df['current_trend_predict'])
location_trend_predict = np.array(feature_df['location_trend_predict'])
runs_scored = np.array(feature_df['runs_scored'])

In [145]:
mape(runs_scored,opponent_trend_predict),mape(runs_scored,current_trend_predict),mape(runs_scored,location_trend_predict)

(33.14884160652851, 31.552384325510975, 36.28616640575539)

In [146]:
max_predict = np.max(np.array(feature_df[['opponent_trend_predict','current_trend_predict','location_trend_predict']]),axis=1)
min_predict = np.min(np.array(feature_df[['opponent_trend_predict','current_trend_predict','location_trend_predict']]),axis=1)
mean_predict = np.mean(np.array(feature_df[['opponent_trend_predict','current_trend_predict','location_trend_predict']]),axis=1)


In [147]:
mape(runs_scored,max_predict),mape(runs_scored,min_predict),mape(runs_scored,mean_predict)

(39.83091281447402, 32.73509820997395, 28.048566843159747)